# Data Prep

By Kenneth Burchfiel

Released under the MIT License

Before a dataset can be analyzed and visualized within Python, it often needs to be reformatted and cleaned. In order to demonstrate how Python can easily perform these reformatting and cleaning tasks, this script will clean and reformat our NVCU winter student survey results; combine those results with fall and spring survey data; and merge in information from our current enrollment table.

Our survey_results database file already contains student survey responses for the fall and spring. However, let's say that you've been asked to add a set of winter results to this dataset as well, then calculate a weighted average of fall, winter, and spring survey results for each student. 

If these results were in the same format as the fall and spring ones and had no missing data, this process would be very simple. Unfortunately, that's not the case with the winter results that we'll be processing within this script. These results feature:

1. Column names that differ from those in the fall/spring results
2. Different data formats
3. A missing column
4. Duplicate values
5. Missing values for certain students
6. Results spread over 16 separate files (one for each school/level pair)

In addition, to make matters even more complex, these winter results are spread out over 16 different files (one for each level within each college).

It would be cumbersome and mind-numbing to modify each of these 16 datasets within Excel, Google Sheets, or a similar program so that they could be combined with our pre-existing fall and spring data. However, the Python code shown below will make this data cleaning process much easier. And once this script is in place, if you happened to get next year's winter results in the same format* as this year's, you'd be able to get them cleaned up and reformatted in no time.

*\*You may find in your work, however, that the results are in yet another format the following year, followed by a different format the year after that. Data-related tasks are always made easier when inputs stay the same, but in the real world, you'll often need to rework datasets in order to make them compatible with pre-existing processes.*

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine

# Cleaning and reformatting winter survey data

Our first step in preparing our winter survey results will be to import the 16 files that comprise them into a DataFrame. We'll first use os.listdir() to create a list of all files within our winter_results folder:

In [2]:
file_list = os.listdir('winter_results')
file_list

['STB_Fr_results.csv',
 'STB_Ju_results.csv',
 'STB_Se_results.csv',
 'STB_So_results.csv',
 'STC_Fr_results.csv',
 'STC_Ju_results.csv',
 'STC_Se_results.csv',
 'STC_So_results.csv',
 'STL_Fr_results.csv',
 'STL_Ju_results.csv',
 'STL_Se_results.csv',
 'STL_So_results.csv',
 'STM_Fr_results.csv',
 'STM_Ju_results.csv',
 'STM_Se_results.csv',
 'STM_So_results.csv']

Next, we'll use a for loop to read each file within this list into a DataFrame. We'll then apply pd.concat() to combine these results into a single DataFrame.

In [3]:
df_list = []
for file in file_list:
    df = pd.read_csv(f'winter_results/{file}')
    df_list.append(df)
df_winter_results = pd.concat(
    [df for df in df_list]) # df for df in df_list is a list comprehension
# that contains all DataFrames in df_list.
df_winter_results.reset_index(drop=True,inplace=True)
df_winter_results

,MATRIC#,SEASON,SURVEY_SCORE,STARTINGYR,MATRICYR
0,25,W,68.0%,23,20
1,1217,W,70.0%,23,20
2,1510,W,61.0%,23,20
3,3027,W,75.0%,23,20
4,1045,W,64.0%,23,20
...,...,...,...,...,...
14617,1008,W,65.0%,23,21
14618,2775,W,78.0%,23,21
14619,776,W,60.0%,23,21
14620,3512,W,56.0%,23,21


The following cell shows a more concise means of creating the same DataFrame. Although this approach requires fewer lines of code, it's also less flexible (as the former method allows you to make individual updates to each DataFrame if needed).

In [4]:
df_winter_results = pd.concat(
    [pd.read_csv(f'winter_results/{file}') 
     for file in os.listdir('winter_results')]).reset_index(drop=True)
df_winter_results

,MATRIC#,SEASON,SURVEY_SCORE,STARTINGYR,MATRICYR
0,25,W,68.0%,23,20
1,1217,W,70.0%,23,20
2,1510,W,61.0%,23,20
3,3027,W,75.0%,23,20
4,1045,W,64.0%,23,20
...,...,...,...,...,...
14617,1008,W,65.0%,23,21
14618,2775,W,78.0%,23,21
14619,776,W,60.0%,23,21
14620,3512,W,56.0%,23,21


## Reformatting and cleaning our dataset

Our next step is to combine these winter survey results with the fall and spring results in our NVCU database. Here's what those results look like:

In [5]:
# Connecting to our database:
e = create_engine('sqlite:///../Appendix/nvcu_db.db')
df_fall_spring_results = pd.read_sql(
    "Select * from survey_results", con = e)
df_fall_spring_results

,student_id,starting_year,season,score
0,2020-1,2023,Fall,88
1,2020-2,2023,Fall,37
2,2020-3,2023,Fall,54
3,2020-4,2023,Fall,56
4,2020-5,2023,Fall,77
...,...,...,...,...
32763,2023-5439,2023,Spring,90
32764,2023-5440,2023,Spring,100
32765,2023-5441,2023,Spring,89
32766,2023-5442,2023,Spring,64


If we naively tried to add our winter results to our fall/spring results, we'd end up with a very messy DataFrame with numerous blank cells:

In [6]:
pd.concat([df_fall_spring_results, df_winter_results])

,student_id,starting_year,season,score,MATRIC#,SEASON,SURVEY_SCORE,STARTINGYR,MATRICYR
0,2020-1,2023.0,Fall,88.0,NaN,NaN,NaN,NaN,NaN
1,2020-2,2023.0,Fall,37.0,NaN,NaN,NaN,NaN,NaN
2,2020-3,2023.0,Fall,54.0,NaN,NaN,NaN,NaN,NaN
3,2020-4,2023.0,Fall,56.0,NaN,NaN,NaN,NaN,NaN
4,2020-5,2023.0,Fall,77.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
14617,NaN,NaN,NaN,NaN,1008.0,W,65.0%,23.0,21.0
14618,NaN,NaN,NaN,NaN,2775.0,W,78.0%,23.0,21.0
14619,NaN,NaN,NaN,NaN,776.0,W,60.0%,23.0,21.0
14620,NaN,NaN,NaN,NaN,3512.0,W,56.0%,23.0,21.0


This messy output is caused by discrepancies in column names between the two tables. We'll need to rename our winter results fields to match their corresponding fields within the fall/spring table. Thankfully, Pandas makes this process very straightforward:

In [7]:
df_winter_results.rename(columns = {
    'SEASON':'season','STARTINGYR':'starting_year',
    'SURVEY_SCORE':'score'}, inplace = True)
df_winter_results

,MATRIC#,season,score,starting_year,MATRICYR
0,25,W,68.0%,23,20
1,1217,W,70.0%,23,20
2,1510,W,61.0%,23,20
3,3027,W,75.0%,23,20
4,1045,W,64.0%,23,20
...,...,...,...,...,...
14617,1008,W,65.0%,23,21
14618,2775,W,78.0%,23,21
14619,776,W,60.0%,23,21
14620,3512,W,56.0%,23,21


We'll also need to convert our `MATRIC#` and `MATRICYR` fields into a single `student_id` field. (This student_id field simply combines students' matriculation years with their matriculation numbers; see nvcu_db_gen.ipynb within the Appendix for more details.) This can be done as follows:

In [8]:
df_winter_results['MATRICYR'] += 2000 # Converts our MATRICYR
# values from YY to YYYY format so that they'll match the format of the 
# matriculation year component of the student_id values within 
# df_fall_spring_results

# Converting students' MATRICYR and MATRIC# values into student IDs:
# (Note that both columns must be converted to strings in order for
# this code to work.)
df_winter_results['student_id'] = (
    df_winter_results['MATRICYR'].astype('str') 
    + '-' 
    + df_winter_results['MATRIC#'].astype('str'))
# Now that we've used our MATRICYR and MATRIC# columns to create 
# our student IDs, we no longer need to retain those columns:
df_winter_results.drop(
    ['MATRICYR', 'MATRIC#'], 
    axis = 1, inplace = True)
df_winter_results

,season,score,starting_year,student_id
0,W,68.0%,23,2020-25
1,W,70.0%,23,2020-1217
2,W,61.0%,23,2020-1510
3,W,75.0%,23,2020-3027
4,W,64.0%,23,2020-1045
...,...,...,...,...
14617,W,65.0%,23,2021-1008
14618,W,78.0%,23,2021-2775
14619,W,60.0%,23,2021-776
14620,W,56.0%,23,2021-3512


The columns in df_winter_results now match those within df_fall_spring_results. That's great! Let's try combining the two datasets to see if we're ready to perform analyses on them:

In [9]:
df_results = pd.concat([df_fall_spring_results, 
           df_winter_results])
df_results.head()

,student_id,starting_year,season,score
0,2020-1,2023,Fall,88
1,2020-2,2023,Fall,37
2,2020-3,2023,Fall,54
3,2020-4,2023,Fall,56
4,2020-5,2023,Fall,77


This output shows that, unfortunately, we're not quite ready to analyze this data just yet: there are several formatting differences that we'll need to address. 

For instance, the 'score' column within df_fall_spring_results uses an integer format, whereas these same numbers are formatted as strings within df_winter_results. This will produce errors when we attempt to perform numerical calculations on this field:

In [10]:
# df_results['score'].mean() 
# Raises a TypeErorr: "unsupported operand type(s) for +: 'int' and 'str'"

The following cell resolves this issue by converting our string-formatted score values to integers:

In [11]:
# Converting our score values to integers:
df_winter_results['score'] = df_winter_results[
'score'].str.replace('.0%','').astype('int')
df_winter_results.head()

,season,score,starting_year,student_id
0,W,68,23,2020-25
1,W,70,23,2020-1217
2,W,61,23,2020-1510
3,W,75,23,2020-3027
4,W,64,23,2020-1045


We'll also need to reformat our winter results' `starting_year` and `season` values so that they match the formats found in the fall/spring table. 

The following cell replaces the 'W' values within the 'season' column with 'Winter' so that they'll match how seasons are formatted within df_fall_spring_results:

In [12]:
df_winter_results['season'] = (
    df_winter_results['season'].replace({'W':'Winter'}))
df_winter_results

,season,score,starting_year,student_id
0,Winter,68,23,2020-25
1,Winter,70,23,2020-1217
2,Winter,61,23,2020-1510
3,Winter,75,23,2020-3027
4,Winter,64,23,2020-1045
...,...,...,...,...
14617,Winter,65,23,2021-1008
14618,Winter,78,23,2021-2775
14619,Winter,60,23,2021-776
14620,Winter,56,23,2021-3512


The following code would also have worked; however, it assumes that every row within the DataFrame is indeed a winter result. This is the case in our simulated data, but in the real world, some data from other seasons might have leaked in, causing this code to incorrectly reclassify certain results.

In [13]:
# df_winter_results['season'] = 'Winter'

Finally, we'll add 2000 to every starting_year value so that our years will show up within YYYY format--just as they do within our fall and spring results.

In [14]:
df_winter_results['starting_year'] += 2000
df_winter_results.head()

,season,score,starting_year,student_id
0,Winter,68,2023,2020-25
1,Winter,70,2023,2020-1217
2,Winter,61,2023,2020-1510
3,Winter,75,2023,2020-3027
4,Winter,64,2023,2020-1045


## Removing duplicates

We've now successfully made our winter dataset's field names and values compatible with those in our fall/spring dataset. However, before we can combine the two together, we'll need to remove some duplicate results.

The following code filters df_winter_results to include any rows whose `season`, `starting_year`, and `student_id` columns match. (The inclusion of `keep = False` instructs Pandas to return all copies of a duplicated row, not just the first one that it encounters.)

In [15]:
df_winter_results[df_winter_results.duplicated(
    subset = ['season', 'starting_year', 'student_id'], 
    keep = False)].head()

,season,score,starting_year,student_id
19,Winter,56,2023,2020-540
68,Winter,63,2023,2020-3122
103,Winter,48,2023,2020-1049
128,Winter,81,2023,2020-1460
154,Winter,46,2023,2020-2327


These duplicate values can easily be removed using Pandas' drop_duplicates() function. However, before removing duplicate rows, it's a good idea to consider which one to retain and then sort the DataFrame accordingly. 

In our case, we'll keep the duplicated row with the highest survey result and remove all others. We can do this by (1) sorting our DataFrame to show higher scores before lower ones and then (2) keeping the first row (e.g. the one with the highest score) when removing our duplicates.

In [16]:
df_winter_results.sort_values(
    'score', ascending = False, inplace = True)
df_winter_results.head()

,season,score,starting_year,student_id
10385,Winter,99,2023,2020-616
65,Winter,98,2023,2020-35
4113,Winter,97,2023,2020-395
13971,Winter,97,2023,2021-468
3729,Winter,97,2023,2020-816


Removing duplicate values:

Note: when removing duplicates, think carefully about which columns to include in your `subset` argument. For instance, if we had multiple years' worth of data in our table, using `['season', 'student_id']` as your subset would cause only *one* result for each student/season pair to get retained, thus removing valid data for other years from your table.

In [17]:
df_winter_results.drop_duplicates(
    subset = ['season', 'starting_year', 'student_id'], 
    keep = 'first', inplace = True)
df_winter_results.head()

,season,score,starting_year,student_id
10385,Winter,99,2023,2020-616
65,Winter,98,2023,2020-35
4113,Winter,97,2023,2020-395
13971,Winter,97,2023,2021-468
3729,Winter,97,2023,2020-816


Rerunning our duplicate check code confirms that no duplicate entries remain within our dataset:

In [18]:
df_winter_results[df_winter_results.duplicated(
    subset = ['season', 'starting_year', 'student_id'], 
    keep = False)].head()

,season,score,starting_year,student_id


## Combining winter survey results with our fall/spring dataset

We're now finally ready to combine df_winter_results with df_fall_spring results. However, one final issue remains with this table, however: winter survey results are missing for a number of students. This won't cause any issues with the following code, but we'll need to take these missing entries into account when analyzing our survey data within descriptive_stats.ipynb.

In [19]:
df_results = pd.concat(
    [df_fall_spring_results, 
     df_winter_results]).sort_values(
    ['starting_year', 'season']).reset_index(drop=True)
df_results

,student_id,starting_year,season,score
0,2020-1,2023,Fall,88
1,2020-2,2023,Fall,37
2,2020-3,2023,Fall,54
3,2020-4,2023,Fall,56
4,2020-5,2023,Fall,77
...,...,...,...,...
46689,2022-2441,2023,Winter,29
46690,2020-2,2023,Winter,28
46691,2023-1318,2023,Winter,28
46692,2023-615,2023,Winter,27


Note that, when our seasons are sorted alphabetically, Fall will come first, followed by Spring and then Winter. In order to allow for a chronological sort (which will prove useful when creating charts and pivot tables), we'll add in a 'season_order' column that maps these seasons to integers.

In [20]:
df_results['season_order'] = df_results['season'].map(
    {'Fall':0,'Winter':1,'Spring':2})
df_results

,student_id,starting_year,season,score,season_order
0,2020-1,2023,Fall,88,0
1,2020-2,2023,Fall,37,0
2,2020-3,2023,Fall,54,0
3,2020-4,2023,Fall,56,0
4,2020-5,2023,Fall,77,0
...,...,...,...,...,...
46689,2022-2441,2023,Winter,29,1
46690,2020-2,2023,Winter,28,1
46691,2023-1318,2023,Winter,28,1
46692,2023-615,2023,Winter,27,1


## Merging college and level data into our survey results table

df_results allows us to calculate survey results at the university-wide level. However, in order to determine whether these results differ by college and by level, we'll need to merge college and level data into our dataset. We can accomplish this by reading data from our curr_enrollment SQL table into a DataFrame, then merging that data with df_results. (The student_id field present in both DataFrames can serve as a merge key.)

*Note: if we had survey data for multiple years, our current enrollment table wouldn't be a good candidate for our merge, since it would show only the current college and levels for each student. We'd instead want to source our enrollment data from a historical enrollment table so that students' yearly survey results could be matched with their levels and colleges during the years that they took the survey.*

In [21]:
# The following code reads in only the fields from df_curr_enrollment 
# that we'll need for our analyses. (If we wanted to read in all fields,
# we could replace the field names in our SQL query with *.)

df_curr_enrollment = pd.read_sql(
    "Select student_id, college, level, \
level_for_sorting from curr_enrollment", con = e)
df_curr_enrollment.head()

,student_id,college,level,level_for_sorting
0,2020-1,STC,Fr,0
1,2020-2,STM,Fr,0
2,2020-3,STC,Fr,0
3,2020-4,STC,Fr,0
4,2020-5,STM,Fr,0


The cell below uses a left merge to add enrollment data into df_results. The `left` argument for the `how` parameter instructs the code to retain all rows in df_results even if no corresponding enrollment data was found. (If we had instead used `right` as our argument, all rows in df_curr_enrollment would have been retained whether or not we had corresponding survey data for them.)

Other options for the `how` parameter include `outer`, which preserves all rows in *both* datasets regardless of whether or not a given key was found in both of them, and `inner`, which would only keep rows whose student IDs were found in both datasets.

In [22]:
df_results = df_results.merge(
    df_curr_enrollment, on = 'student_id', how = 'left')
df_results.head()

,student_id,starting_year,season,score,season_order,college,level,level_for_sorting
0,2020-1,2023,Fall,88,0,STC,Fr,0
1,2020-2,2023,Fall,37,0,STM,Fr,0
2,2020-3,2023,Fall,54,0,STC,Fr,0
3,2020-4,2023,Fall,56,0,STC,Fr,0
4,2020-5,2023,Fall,77,0,STM,Fr,0


Now that we've merged in college and level data, we'll save this dataset to a .csv file so that it can be processed by descriptive_stats.ipynb:

In [23]:
df_results.to_csv('2023_survey_results.csv', index = False)

This script has provided an introduction to data cleaning and reformatting. Other PFN sections will provide further examples of data reformatting, as reshaping data is often a necessary prerequisite for analysis and visualization tasks.